# Match BGBM Collectors to Wikidata Items Using *Cosine Similarity*, `eventDate` Involved

In this example we add `eventDate` of the source data, when the sample/occurrence was collected, to have a time reference, when the collector should have been  alive.

Basically we …

- match of `canonical_string` of WikiData to `canonical_string` of the source collectors (abbreviated names and full names, if given), and
- parse collector source names beforehand to get individual names out of name lists in the source data, we have used <https://libraries.io/rubygems/dwc_agent>, and in general we
- follow the example of Niels Klazenga <https://github.com/nielsklazenga/avh-collectors/blob/master/cosine_similarity.ipynb>
- write the output to provide a DarwinCore attribution structure (for `verbatimName` we would need the `source_data` name(s))

Technical Notes — Review Code perhaps:
- TODO review score calculation of the matching of relating eventData with range of yob, yod
- TODO review DwC agent output, keep at this time custom columns for filter-sort-evaluation convenience
- (NN ⇌ Cosine) refactor relation: wd_matchtest ⇌ wikidata_unique (replaced wd_matchtest → wikidata_unique)

### Load Wikidata Data Set

Construct data using Jupyter Notebook [create_wikidata_datasets_botanists.ipynb](./create_wikidata_datasets_botanists.ipynb)

Out of the Wikidata items data set we create a data frame with unique canonical name strings and their counts.

In [1]:
import pandas as pd
import pprint, time, os

wikidata = pd.read_csv(
    # "data/wikidata_persons_botanists_20231030_1539.csv", # inverse match: [particle +] family, given
    "data/wikidata_persons_botanists_20231116.csv",        # match: given [+ particle] + family[+ , suffix]
    index_col=0, low_memory=False,
    dtype={
        'yob':'Int32',
        'yod':'Int32',
        'wyb':'Int32',
        'wye':'Int32'
    }    
)
# # # # # # # # # # # # # # # # 
# def convert_to_datetime(year):
#     year = int(year)
#     days = int((year - int(year)) * 365.25)
#     base_date = pd.to_datetime(f'{year}-01-01', errors="ignore")
#     # ,format='%Y-%m-%d', errors='coerce'
#     # base_date = pd.Period(year, freq='D')
#     return base_date + pd.DateOffset(days=days)
# 
# def convert_to_datetime(year):
#     base_date = pd.to_datetime(f'{year}-01-01', errors="ignore")
# print(wikidata.dtypes)

# wikidata['yob_converted'] = wikidata['yob'].apply(convert_to_datetime)
# wikidata['yod_converted'] = wikidata['yod'].apply(lambda x: pd.Period(x, freq='Y')) # Given date string "-286" not likely a datetime
# # # # # # # # # # # # # # # # 

pprint.pprint(wikidata.columns)
display(wikidata.head())

Index(['item', 'itemLabel', 'surname', 'initials', 'canonical_string',
       'canonical_string_fullname', 'orcid', 'viaf', 'isni', 'harv', 'ipni',
       'abbr', 'bionomia_id', 'yob', 'yod', 'wyb', 'wye', 'wikidata_link',
       'orcid_link', 'harv_link', 'ipni_link', 'bionomia_link'],
      dtype='object')


,item,itemLabel,surname,initials,canonical_string,canonical_string_fullname,orcid,viaf,isni,harv,...,bionomia_id,yob,yod,wyb,wye,wikidata_link,orcid_link,harv_link,ipni_link,bionomia_link
0,http://www.wikidata.org/entity/Q66612,Friedrich August Marschall von Bieberstein,Friedrich August Marschall von,F. A. M. v.,F. A. M. v. Bieberstein,Friedrich August Marschall von Bieberstein,NaN,43340073,0000 0001 1630 5464,1373,...,Q66612,1768,1826,<NA>,<NA>,http://www.wikidata.org/wiki/Q66612,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/6129-1,https://bionomia.net/Q66612
1,http://www.wikidata.org/entity/Q66934,Hans Hermann Behr,Hans Hermann,H. H.,H. H. Behr,Hans Hermann Behr,NaN,20328622,0000 0001 1604 8680,42741,...,Q66934,1818,1904,<NA>,<NA>,http://www.wikidata.org/wiki/Q66934,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/619-1,https://bionomia.net/Q66934
2,http://www.wikidata.org/entity/Q66661,Jacob Christian Schäffer,Jacob Christian,J. C.,J. C. Schäffer,Jacob Christian Schäffer,NaN,47016953,0000 0000 8343 3899,1101,...,NaN,1718,1790,<NA>,<NA>,http://www.wikidata.org/wiki/Q66661,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/12818-1,NaN
3,http://www.wikidata.org/entity/Q67003,Johann Friedrich Klotzsch,Johann Friedrich,J. F.,J. F. Klotzsch,Johann Friedrich Klotzsch,NaN,20426762,0000 0001 1749 2732,135,...,Q67003,1805,1860,<NA>,<NA>,http://www.wikidata.org/wiki/Q67003,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/4855-1,https://bionomia.net/Q67003
4,http://www.wikidata.org/entity/Q66322,Franz Anton Menge,Franz Anton,F. A.,F. A. Menge,Franz Anton Menge,NaN,59847236,0000 0001 1653 0899,73782,...,NaN,1808,1880,<NA>,<NA>,http://www.wikidata.org/wiki/Q66322,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/23266-1,NaN


In [2]:
# compile data having only unique canonical strings
# group by canonical name/string, count douplicated names
wd_matchtest = wikidata.groupby('canonical_string').agg({'item': ['count']}).reset_index()
wd_matchtest_fullnames = wikidata.groupby('canonical_string_fullname').agg({'item': ['count']}).reset_index()

display(wd_matchtest)
display(wd_matchtest_fullnames)


,canonical_string,item
,,count
0,('W.') S. W. Wong,1
1,(A. A.) G. L. Monnier,1
2,(A.) F. Valet,1
3,(A.) H. (S.) Stenar,1
4,(A.) T. Wegelin,1
...,...,...
69099,Э. Э. Керн,1
69100,Ю. К. Шель,1
69101,Ю. П. Нюкша,1


,canonical_string_fullname,item
,,count
0,('Wilson') Sze Wing Wong,1
1,(Alexandre Alexis) George Le Monnier,1
2,(Antonius) Theodoor Wegelin,1
3,(August) Friedrich Valet,1
4,(Axel) Helge (Svensson) Stenar,1
...,...,...
71394,Эдуард Эдуардович Керн,1
71395,Юлиан Карлович Шель,1
71396,Юлия Петровна Нюкша,1


## Load Collectors Data Set

**Data sources:**

- option 1: Jupyter Notebook for [create_bgbm_gbif-occurrence_collectors_dataset.ipynb](./create_bgbm_gbif-occurrence_collectors_dataset.ipynb)
- option 2: Jupyter Notebook for `create_bgbm_botanypilot_collectors_dataset.ipynb` from SPARQL (not in this official documentation yet)

Then parse collector names to get single, separate collector names using `dwcagent`, use ruby gem package available at  <https://rubygems.org/gems/dwc_agent>:

- see [./bin/README.md](bin/README.md) to use ruby script `./bin/agent_parse4tsv.rb` for parsing text lines like `"Abbe,L.B., Abbe,E.C., Smitinand,T. & Rollet,B."`

Technical notes:

- the corresponding objects, variable names of Nils’ python code were:
```
refactor df_avh = → = collectors
refactor df_avh['label'] = → = collectors['canonical_string_collector_parsed']
…
```

In [3]:
# unique names parsed already by ruby gem package: dwcagent

# collectors = pd.read_csv("data/bgbm_collectors_20230510_1429_single-line_parsed_unique_names.tab", sep="\t")
collectors = pd.read_csv("data/VHde_doi-10.15468-dl.tued2e/occurrence_recordedBy_eventDate_occurrenceIDs_20230823_parsed.tsv", sep="\t", low_memory=False)

collectors.dropna(subset=['family'], inplace=True) # remove where family was NA, e.g. from originally «??» aso.

print("modify time using pd.Periode(…) to get it work also on very old dates...")
for col in ['eventDate_mean', 'eventDate_min', 'eventDate_max']:
    print("- convert", col, "to pd.Period(...) in collectors")
    collectors[col] = collectors[col].apply(
        lambda x: pd.Period(
            x, freq='S' if col.lower().endswith('mean') else 'D' # Seconds or Day level
        )
    ) # D=day level
    # see https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-period-aliases
print("modifing done.")

collectors.sort_values(by=['family', 'given','occurrenceID_firstsample'], inplace=True)
collectors

modify time using pd.Periode(…) to get it work also on very old dates...
- convert eventDate_mean to pd.Period(...) in collectors
- convert eventDate_min to pd.Period(...) in collectors
- convert eventDate_max to pd.Period(...) in collectors
modifing done.


,family,given,suffix,particle,dropping_particle,nick,appellation,title,source_data,parsed_names,cleaned_names,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max
2,A. Cano,E.,NaN,NaN,NaN,NaN,NaN,NaN,"A. Cano,E.",parsed:E. A. Cano,cleaned:E. A. Cano,1,https://herbarium.bgbm.org/object/B100699397,2008-06-05 00:00:00,2008-06-05,2008-06-05
31620,Aaiki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Murashige & Aaiki,parsed:Murashige<SEP>Aaiki,cleaned:Murashige<SEP>Aaiki,1,https://herbarium.bgbm.org/object/B101149305,1949-03-02 00:00:00,1949-03-02,1949-03-02
5,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Aaronsohn,A.",parsed:A. Aaronsohn,cleaned:A. Aaronsohn,1,https://herbarium.bgbm.org/object/B100379341,1908-06-20 00:00:00,1908-06-20,1908-06-20
22504,Abaouz,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Jury,S.L., Abaouz,A. ,Lafkih,M.Ait & Griffiths...",parsed:S.L. Jury<SEP>A. Abaouz<SEP>M.Ait Lafki...,cleaned:S.L. Jury<SEP>A. Abaouz<SEP>M. Ait Laf...,3,https://herbarium.bgbm.org/object/B100217620,1997-07-10 16:00:00,1997-07-05,1997-07-14
22508,Abaouz,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Jury,S.L., Abaouz,A., Ait Lafkih,M. & Griffith...",parsed:S.L. Jury<SEP>A. Abaouz<SEP>M. Ait Lafk...,cleaned:S.L. Jury<SEP>A. Abaouz<SEP>M. Ait Laf...,2,https://herbarium.bgbm.org/object/B100326682,1997-07-09 12:00:00,1997-07-07,1997-07-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12973,Файвуш,Г.,NaN,NaN,NaN,NaN,NaN,NaN,"Fayvush,G. [Ֆայվուշի,Գ.; Файвуш Г.], Pignotti,...",parsed:G.Ֆայվուշի Fayvush<SEP>Գ.<SEP>Г. Файвуш...,cleaned:G.Ֆայվուշի Fayvush<SEP>Գ.<SEP>Г. Файву...,5,https://herbarium.bgbm.org/object/B100513446,2010-06-13 00:00:00,2010-06-13,2010-06-13
12980,Файвуш,Г.,NaN,NaN,NaN,NaN,NaN,NaN,"Fayvush,G. [Ֆայվուշի,Գ.; Файвуш Г.], Tamanyan,...",parsed:G.Ֆայվուշի Fayvush<SEP>Գ.<SEP>Г. Файвуш...,cleaned:G.Ֆայվուշի Fayvush<SEP>Գ.<SEP>Г. Файву...,4,https://herbarium.bgbm.org/object/B100559866,2008-03-02 06:00:00,2004-06-03,2009-06-02
12941,Файвуш,Г.,NaN,NaN,NaN,NaN,NaN,NaN,"Fayvush,G. [Ֆայվուշի,Գ.; Файвуш Г.], Oganesian...",parsed:G.Ֆայվուշի Fayvush<SEP>Գ.<SEP>Г. Файвуш...,cleaned:G.Ֆայվուշի Fayvush<SEP>Գ.<SEP>Г. Файву...,7,https://herbarium.bgbm.org/object/B100810598,2015-07-07 00:00:00,2015-07-04,2015-07-09
14284,Գաբրիելյան,Gabrielian,NaN,NaN,NaN,NaN,NaN,NaN,Gabrielian [Գաբրիելյան; Габриелян] & Dittrich,parsed:Gabrielian Գաբրիելյան<SEP>Габриелян<SEP...,cleaned:Gabrielian Գաբրիելյան<SEP>Габриелян<SE...,1,https://herbarium.bgbm.org/object/B100088191,1989-07-12 00:00:00,1989-07-12,1989-07-12


### Check Composition of Parsed Collector Data

In [4]:
# TODO review code of abbreviated names and full name matching
criterion_fullnames = collectors.given.str.contains('^\w{3,}', na=False)
print("Show collecors given name has (propably) a full name (%s records) …" % len(collectors[criterion_fullnames].index))
collectors[criterion_fullnames]

Show collecors given name has (propably) a full name (606 records) …


,family,given,suffix,particle,dropping_particle,nick,appellation,title,source_data,parsed_names,cleaned_names,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max
2023,Abdallah,Raffael,NaN,NaN,NaN,NaN,NaN,NaN,"Bally,P.R.O., Abdallah, Raffael & Reichstein, T.",parsed:P.R.O. Bally<SEP>Raffael Abdallah<SEP>T...,cleaned:P.R.O. Bally<SEP>Raffael Abdallah<SEP>...,1,https://herbarium.bgbm.org/object/B200125981,1969-11-18 00:00:00,1969-11-18,1969-11-18
22587,Abdul,Kadir Bin,NaN,NaN,NaN,NaN,NaN,NaN,Kadir Bin Abdul,parsed:Kadir Bin Abdul,cleaned:Kadir Bin Abdul,1,https://herbarium.bgbm.org/object/B100184021,1950-03-20 00:00:00,1950-03-20,1950-03-20
17278,Abreu,Guilherme,NaN,de,NaN,NaN,NaN,NaN,Guilherme de Abreu (no. 103),parsed:Guilherme de Abreu,cleaned:Guilherme de Abreu,1,http://id.snsb.info/snsb/collection/22086/3086...,1938-08-24 00:00:00,1938-08-24,1938-08-24
15125,Adá,García,NaN,NaN,NaN,NaN,NaN,NaN,"García Adá, Luceño, Rico,E., Romero,T. & Varga...",parsed:García Adá<SEP>Rico Luceño<SEP>E. Romer...,cleaned:García Adá<SEP>Rico Luceño<SEP>E. Rome...,1,https://herbarium.bgbm.org/object/B100296455,1987-06-19 00:00:00,1987-06-19,1987-06-19
399,Ahagen,Schiers,NaN,NaN,NaN,NaN,NaN,NaN,"Ahagen, Schiers,C. & al.",parsed:Schiers Ahagen<SEP>C.<SEP>Al,cleaned:Schiers Ahagen<SEP>C.<SEP>Al,1,https://herbarium.bgbm.org/object/B100194646,1987-07-03 00:00:00,1987-07-03,1987-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30642,Мирзоева,Mirzoeva,NaN,NaN,NaN,NaN,NaN,NaN,Mirzoeva [Мирзоева] & Gambarian & Pogosian,parsed:Mirzoeva Мирзоева<SEP>Gambarian<SEP>Pog...,cleaned:Mirzoeva Мирзоева<SEP>Gambarian<SEP>Po...,1,https://herbarium.bgbm.org/object/B100355321,1960-03-30 00:00:00,1960-03-30,1960-03-30
34734,Попов,Popov,NaN,NaN,NaN,NaN,NaN,NaN,Popov [Попов],parsed:Popov Попов,cleaned:Popov Попов,3,https://dr.jacq.org/DR049416,1924-09-28 00:00:00,1923-06-17,1925-05-30
34735,Попов,Popov,NaN,NaN,NaN,NaN,NaN,NaN,Popov [Попов] & Vvedensky,parsed:Popov Попов<SEP>Vvedensky,cleaned:Popov Попов<SEP>Vvedensky,4,https://herbarium.bgbm.org/object/B101149435,1923-08-27 06:00:00,1923-05-15,1924-05-22
14284,Գաբրիելյան,Gabrielian,NaN,NaN,NaN,NaN,NaN,NaN,Gabrielian [Գաբրիելյան; Габриелян] & Dittrich,parsed:Gabrielian Գաբրիելյան<SEP>Габриелян<SEP...,cleaned:Gabrielian Գաբրիելյան<SEP>Габриелян<SE...,1,https://herbarium.bgbm.org/object/B100088191,1989-07-12 00:00:00,1989-07-12,1989-07-12


In [5]:
# check the name-parsed columns if they are empty or need to be considerd as data for matching or not
for parsed_name_part in ["particle", "suffix", "dropping_particle", "appellation"]:
    test_collectors = collectors.loc[(collectors[parsed_name_part].isna() == False)]
    print("\n----------------------------------------\nshow names with **%s** found %s records:\n" % (parsed_name_part, len(test_collectors.index)))
    display(test_collectors.head().get(["family", "given", "suffix", "particle", "dropping_particle", "nick", "appellation", "title"]))    


----------------------------------------
show names with **particle** found 603 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
17278,Abreu,Guilherme,NaN,de,NaN,NaN,NaN,NaN
347,Aghababyan,M.,NaN,von,NaN,NaN,NaN,NaN
3481,Aguilar,M.L.,NaN,Reyna de,NaN,NaN,NaN,NaN
49605,Aguilar,M.L.,NaN,Reyna de,NaN,NaN,NaN,NaN
13649,Aguilar,M.L.,NaN,Reyna de,NaN,NaN,NaN,NaN



----------------------------------------
show names with **suffix** found 14 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
13999,August,Friedrich,II.,NaN,NaN,NaN,NaN,NaN
47886,Dogma,I.J.,Jr.,NaN,NaN,NaN,NaN,NaN
13785,Forsyth,W.,jr.,NaN,NaN,NaN,NaN,NaN
769,Grear,J.W.,Jr.,NaN,NaN,NaN,NaN,NaN
21795,Grear,J.W.,Jr.,NaN,NaN,NaN,NaN,NaN



----------------------------------------
show names with **dropping_particle** found 0 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title



----------------------------------------
show names with **appellation** found 1 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
13850,Sennen,NaN,NaN,NaN,NaN,NaN,Fr,NaN


Compile `canonical_string…` for the collector data we will later match the WikiData names with:

In [6]:
# combine parts of names similar to WikiData's given name labels
collectors['canonical_string_collector_parsed'] = collectors[['given', 'particle', 'family', 'suffix']]\
    .fillna('')\
    .apply(
        lambda this_df: "{given}{particle}{family}{suffix}".format(
            given=this_df["given"],
            particle=" " + this_df["particle"] if this_df["particle"] else '', 
            family=" " + this_df["family"] if this_df["family"] else '', 
            suffix=", " + this_df["suffix"] if this_df["suffix"] else ''
        ), axis="columns"
    )

criterion = collectors["particle"].str.contains("\w+ \w+", na=False)

# display(collectors['canonical_string_collector_parsed'][criterion].head())
collectors[['canonical_string_collector_parsed', 'particle']][criterion].drop_duplicates().head(10)

,canonical_string_collector_parsed,particle
3481,M.L. Reyna de Aguilar,Reyna de
6,H. Abbas al Ani,Abbas al
39610,J. Francisco del Aquila,Francisco del
29014,A.L.M. Marcailhou d' Aymeric,Marcailhou d'
37779,P.R.O. Ritchie in Bally,Ritchie in
33712,H. Perrier de la Bathie,Perrier de la
33163,A.M.F.J. Palisot de Beauvois,Palisot de
49214,D. Van der Ben,Van der
53826,B. van den Boom,van den
53825,P. van den Boom,van den


In [7]:
# move canonical_string_collector_parsed after column title (title was the last of the parsing columns)
col = collectors.pop("canonical_string_collector_parsed")
collectors.insert(collectors.columns.get_loc('title') + 1, col.name, col)

these_columns=["family", "given", "suffix", "particle", "dropping_particle", "nick", "appellation", "title", 'canonical_string_collector_parsed']

if 'source_data' in collectors.columns:
    these_columns.append("source_data")

display(collectors.tail().get(these_columns))

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data
12973,Файвуш,Г.,NaN,NaN,NaN,NaN,NaN,NaN,Г. Файвуш,"Fayvush,G. [Ֆայվուշի,Գ.; Файвуш Г.], Pignotti,..."
12980,Файвуш,Г.,NaN,NaN,NaN,NaN,NaN,NaN,Г. Файвуш,"Fayvush,G. [Ֆայվուշի,Գ.; Файвуш Г.], Tamanyan,..."
12941,Файвуш,Г.,NaN,NaN,NaN,NaN,NaN,NaN,Г. Файвуш,"Fayvush,G. [Ֆայվուշի,Գ.; Файвуш Г.], Oganesian..."
14284,Գաբրիելյան,Gabrielian,NaN,NaN,NaN,NaN,NaN,NaN,Gabrielian Գաբրիելյան,Gabrielian [Գաբրիելյան; Габриелян] & Dittrich
47853,Թամանյան,Tamanian,NaN,NaN,NaN,NaN,NaN,NaN,Tamanian Թամանյան,Tamanian [Թամանյան; Таманян]


In [8]:
# group and aggregate data to have unique name rows only for the matching of names later on
collectors_unique=collectors.groupby(['canonical_string_collector_parsed']).agg(
    family=('family', lambda x: list(x)[0]),
    given=('given', lambda x: list(x)[0]),
    suffix=('suffix', lambda x: list(x)[0]),
    particle=('particle', lambda x: list(x)[0]),
    dropping_particle=('dropping_particle', lambda x: list(x)[0]),
    nick=('nick', lambda x: list(x)[0]),
    appellation=('appellation', lambda x: list(x)[0]),
    title=('title', lambda x: list(x)[0]),
    source_data=('source_data', lambda x: list(x)[0]),
    occurrenceID_collectors_count= ('occurrenceID_count', 'sum'), # use count function
    occurrenceID_collectors_firstsample=('occurrenceID_firstsample', lambda x: list(x)[0]), # custom function, to get the first entry
    collectors_eventDate_mean=('eventDate_mean', 'mean'),
    collectors_eventDate_min=('eventDate_min', 'min'),
    collectors_eventDate_max=('eventDate_max', 'max')
).reset_index()

# move canonical_string_collector_parsed after column title
col = collectors_unique.pop("canonical_string_collector_parsed")
collectors_unique.insert(collectors_unique.columns.get_loc('title') + 1, col.name, col)

display(collectors_unique)

# column naming perhaps more clear (because we condensed the data)?
# collectors=collectors.add_suffix('_namegrouped') \
#  if not any(col.endswith("_namegrouped") for col in list(collectors.columns))

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max
0,Aaiki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aaiki,Murashige & Aaiki,1,https://herbarium.bgbm.org/object/B101149305,1949-03-02 00:00:00,1949-03-02,1949-03-02
1,Abner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abner,Abner,1,https://herbarium.bgbm.org/object/B180021124,2007-07-21 00:00:00,2007-07-21,2007-07-21
2,Abramov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abramov,Abramov,1,https://dr.jacq.org/DR071678,1905-06-01 00:00:00,1905-06-01,1905-06-01
3,Acevedo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Acevedo,"Rodríguez,A., Estrada,A., González,J., Chacón,...",1,https://herbarium.bgbm.org/object/B101131930,2001-03-07 00:00:00,2001-03-07,2001-03-07
4,Achverdov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Achverdov,Mirzoeva [Мирзоева] & Achverdov,12,https://herbarium.bgbm.org/object/B100089402,1955-12-22 20:32:00,1948-03-04,1968-03-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18418,Таманян,К.,NaN,NaN,NaN,NaN,NaN,NaN,К. Таманян,"Tamanian,K. [Թամանյան,Կ.; Таманян,К.]",16,https://herbarium.bgbm.org/object/B100190731,2000-05-29 03:50:24,1984-04-18,2010-06-18
18419,Оганесиан,М.,NaN,NaN,NaN,NaN,NaN,NaN,М. Оганесиан,"Oganesian,M. [Ոգանեսիան,Մ.; Оганесиан,М.], Ter...",25,https://herbarium.bgbm.org/object/B100275643,2005-01-18 17:06:00,1977-06-19,2010-06-11
18420,Androsov,Н.,NaN,NaN,NaN,NaN,NaN,NaN,Н. Androsov,"Androsov, Н. (no. 2404a)",1,http://id.snsb.info/snsb/collection/455833/556...,1911-05-06 00:00:00,1911-05-06,1911-05-06
18421,Габриелян,Н.,NaN,NaN,NaN,NaN,NaN,NaN,Н. Габриелян,"Gabrieljan,N. [Գաբրիելյան,Ն.; Габриелян,Н.] & ...",1,https://herbarium.bgbm.org/object/B100190831,1976-04-08 00:00:00,1976-04-08,1976-04-08


## Set Up the Cosine Similarity and Text Search

See 
- for the application code https://github.com/nielsklazenga/avh-collectors/blob/master/cosine_similarity.ipynb
- for reading on the topic: Taylor, Josh. 2019. ‘Fuzzy Matching at Scale’. Towards Data Science (blog). 2 July 2019. https://towardsdatascience.com/fuzzy-matching-at-scale-84f2bfd0c536.

The `ngrams`-function is used as an analyzer in the text search later.

In [9]:
import pandas as pd, numpy as np, re
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
from sparse_dot_topn import awesome_cossim_topn # pip install sparse-dot-topn

def get_matches_df(sparse_matrix, A, B, top=100):
    non_zeros = sparse_matrix.nonzero()

    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]

    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size

    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similarity = np.zeros(nr_matches)

    for index in range(0, nr_matches):
        left_side[index] = A[sparserows[index]]
        right_side[index] = B[sparsecols[index]]
        similarity[index] = round(sparse_matrix.data[index], 3)

    return pd.DataFrame({'namematch_source_data': left_side,
                         'namematch_resource_data': right_side,
                         'namematch_similarity': similarity})

!pip install ftfy
from ftfy import fix_text

def ngrams(string, n=3):
    """
    Construct ngram(s) of a given text

    @param string: the text string to perform the ngram splitting on
    @param n: character length of the particular (split) result text each
    @return: string as ngram
    """
    string = fix_text(string) # fix text
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower()
    chars_to_remove = [")","(","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.replace('.', ' ')
    string = string.title()  # normalise case - capital at start of each word
    string = re.sub(' +', ' ', string).strip() # get rid of multiple spaces and replace with a single
    string = ' ' + string + ' '  # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD',r'', string)
    string = string.strip()
    this_ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in this_ngrams]

def calculateTFIDFmatchingOfData(query_data, match_data, cossim_ntop=1, cossim_lower_bound=0.5):
    """
    Calculate a TF-IDF (Term Frequency — Inverse Document Frequency) matching with awesome_cossim_topn() and return matched data

    @param query_data: DataFrame usually a pandas data column to query names or strings for
    @param match_data: DataFrame against to match with
    @param cossim_ntop: how many cossim matches each shall be calculated (default 1, i.e. the highest similarity) — increase it to get more alternative
        matches with less similarity
    @param cossim_lower_bound: where is the lower similarity cut off to regard data as similar (default 0.5)

    @requires get_get_matches_df()
    @requires ngrams()
    @requires awesome_cossim_topn()
    @requires TfidfVectorizer()

    @return: a data frame dictionary: namematch_source_data, namematch_resource_data, namematch_similarity (from @see get_matches_df())
    @rtype pd.DataFrame
    """

    import time
    time_start = time.time()

    # Vectorize Wikidata name (use fit_transform())
    print('Vectorizing data. This may take a while...')
    vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
    tf_idf_matrix_clean = vectorizer.fit_transform(match_data)
    # Vectorize collectors’ names (use transform())
    tf_idf_matrix_dirty = vectorizer.transform(query_data)

    duration = time.time() - time_start
    print('Vectorizing completed: created a matrix of TF-IDF features after %s s' % duration)

    # Calculate Cosine Similarity; keep only the best match (ntop=1) and only if the similarity is greater than 0.5 (lower_bound=0.5)
    # (lower_bound: a threshold that the element of A*B must be greater than
    #  https://github.com/ing-bank/sparse_dot_topn/blob/3f40611b0553b50c27f23c7dcffc3ca9a9e8f5b5/sparse_dot_topn/awesome_cossim_topn.py#L26C9-L26C78)
    cossim_matches = awesome_cossim_topn(
        tf_idf_matrix_dirty,
        tf_idf_matrix_clean.transpose(),
        ntop=cossim_ntop,
        lower_bound=cossim_lower_bound
    )
    print("Cossim matches calculated after %s s" % (time.time() - time_start))

    print("Get all matches together ...")
    # construct the matching data frame
    matches_df = get_matches_df(
        cossim_matches,
        query_data,
        match_data,
        top=0
    )
    print("Done. Matches calculated after %s s" % (time.time() - time_start))

    return matches_df

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try 'pacman -S
    python-xyz', where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Arch-packaged Python package,
    create a virtual environment using 'python -m venv path/to/venv'.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip.
    
    If you wish to install a non-Arch packaged Python application,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. Make sure you have python-pipx
    installed via pacman.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detailed specification.


In [10]:
# some example data
print("Show ngram examples:")
print("- simple name:", ngrams('Klazenga, N.'))
print("- data from collectors:", ngrams(collectors_unique["canonical_string_collector_parsed"].at[1])) 
print("- data from match-test:", ngrams(wd_matchtest['canonical_string'].at[0]))
print("- data from match-test (full name):", ngrams(wd_matchtest_fullnames['canonical_string_fullname'].at[0]))

# some example data
for i, row in enumerate(range(5)):
    if (i == 0):
        print('\n(WikiData’s) canonical_string = (constructed) canonical_string_fullname:') 
    print("- {short_name} = {long_name}".format(
        short_name=wd_matchtest['canonical_string'].at[row],
        long_name=wd_matchtest_fullnames['canonical_string_fullname'].at[row]
    ))


Show ngram examples:
- simple name: ['Kla', 'laz', 'aze', 'zen', 'eng', 'nga', 'ga ', 'a N']
- data from collectors: ['Abn', 'bne', 'ner']
- data from match-test: ['W S', ' S ', 'S W', ' W ', 'W W', ' Wo', 'Won', 'ong']
- data from match-test (full name): ['Wil', 'ils', 'lso', 'son', 'on ', 'n S', ' Sz', 'Sze', 'ze ', 'e W', ' Wi', 'Win', 'ing', 'ng ', 'g W', ' Wo', 'Won', 'ong']

(WikiData’s) canonical_string = (constructed) canonical_string_fullname:
- ('W.') S. W. Wong = ('Wilson') Sze Wing Wong
- (A. A.) G. L. Monnier = (Alexandre Alexis) George Le Monnier
- (A.) F. Valet = (Antonius) Theodoor Wegelin
- (A.) H. (S.) Stenar = (August) Friedrich Valet
- (A.) T. Wegelin = (Axel) Helge (Svensson) Stenar


In [11]:
criterion_fullnames = collectors_unique.given.str.contains('^\w{3,}', na=False)
collectors_names = collectors_unique['canonical_string_collector_parsed'][[not fullname for fullname in criterion_fullnames]].values

matches = calculateTFIDFmatchingOfData(
    collectors_names, 
    wd_matchtest['canonical_string'], 
    cossim_ntop=1 # e.g. cossim_ntop=3 would give more alternative matches as well, having lower similarities, data would increase 3 times as well
)
matches = matches.sort_values(by=['namematch_similarity'], ascending=[False])
matches = matches.reset_index(names=['old_index'])
display(matches)

Vectorizing data. This may take a while...
Vectorizing completed: created a matrix of TF-IDF features after 4.12286901473999 s
Cossim matches calculated after 4.581612825393677 s
Get all matches together ...
Done. Matches calculated after 4.732778310775757 s


,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
0,8372,H. Schultze-Rhonhof,H. Schultze-Rhonhof,1.0
1,3748,A.H. Gentry,A. H. Gentry,1.0
2,3756,A.H.G. Alston,A. H. G. Alston,1.0
3,3755,A.H. Wolley-Dod,A. H. Wolley-Dod,1.0
4,3754,A.H. Unwin,A. H. Unwin,1.0
...,...,...,...,...
16017,1611,Mortelmans,A. Morte,0.5
16018,4863,C. von Moor,M. Moor,0.5
16019,1389,Levnhardt,S. Linhardt,0.5
16020,7336,G. Fineman,K. E. Fine,0.5


In [12]:
# criterion_fullnames = collectors_unique.given.str.contains('^\w{3,}', na=False)
collectors_fullnames = collectors_unique['canonical_string_collector_parsed'][criterion_fullnames].values
matches_fullnames = calculateTFIDFmatchingOfData(
    collectors_fullnames, 
    wd_matchtest_fullnames['canonical_string_fullname'], 
    cossim_ntop=1 # 10 would give more alternative matches also with lesser similarity
)

matches_fullnames = matches_fullnames.sort_values(by=['namematch_similarity'], ascending=[False])
matches_fullnames = matches_fullnames.reset_index(names=['old_index'])

display(matches_fullnames)

Vectorizing data. This may take a while...
Vectorizing completed: created a matrix of TF-IDF features after 3.647350549697876 s
Cossim matches calculated after 3.7035231590270996 s
Get all matches together ...
Done. Matches calculated after 3.7072396278381348 s


,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
0,131,Georg Wilhelm Schimper,Georg Wilhelm Schimper,1.000
1,192,Joseph Friedrich Knaf,Joseph Friedrich Knaf,1.000
2,200,Jürg Steiger,Jürg Steiger,1.000
3,199,Justus J.H. Schmidt,Justus J.H. Schmidt,1.000
4,314,Thomas Pichler,Thomas Pichler,1.000
...,...,...,...,...
338,134,Giese Frahm,G. Frahm,0.507
339,63,Clark Mendoza-T.P.,George Mendoza,0.506
340,97,Fedor Alexandrowitsch Teplouchow,Fedor Teploukhov,0.505
341,140,Guymer Sharpe,Gordon P. Guymer,0.504


### Create Output Results

Combine the matches data frame back to the (BGBM) collectors and Wikidata items …

Note: merging 18.770.000 collector matches earlier to wikidata was too much to calculate. Hence the descision was to make the data unique by canonical_string_collector_parsed.

In [13]:
# # join (only) abbreviated name matches with collector source data
collectors_matches = pd.merge(
    collectors_unique, matches, 
    left_on='canonical_string_collector_parsed', right_on='namematch_source_data', 
    how='left'
)

collectors_matches.dropna(subset=['namematch_similarity'], inplace=True) # drop all matches that the method gave a NA (we narrowed the similarity range from 0.5 to 1.0)
collectors_matches # 17552 rows × 15 columns

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
0,Aaiki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aaiki,Murashige & Aaiki,1,https://herbarium.bgbm.org/object/B101149305,1949-03-02 00:00:00,1949-03-02,1949-03-02,0.0,Aaiki,K. Saiki,0.720
1,Abner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abner,Abner,1,https://herbarium.bgbm.org/object/B180021124,2007-07-21 00:00:00,2007-07-21,2007-07-21,1.0,Abner,J. Hübner,0.559
2,Abramov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abramov,Abramov,1,https://dr.jacq.org/DR071678,1905-06-01 00:00:00,1905-06-01,1905-06-01,2.0,Abramov,A. Abramova,0.843
3,Acevedo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Acevedo,"Rodríguez,A., Estrada,A., González,J., Chacón,...",1,https://herbarium.bgbm.org/object/B101131930,2001-03-07 00:00:00,2001-03-07,2001-03-07,3.0,Acevedo,R. Acevedo,0.884
4,Achverdov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Achverdov,Mirzoeva [Мирзоева] & Achverdov,12,https://herbarium.bgbm.org/object/B100089402,1955-12-22 20:32:00,1948-03-04,1968-03-29,4.0,Achverdov,A. A. Achverdov,0.826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18409,Drouhard,é-J.,NaN,NaN,NaN,NaN,NaN,NaN,é-J. Drouhard,"Drouhard,É.-J.",2,https://je.jacq.org/JE04000802,1924-01-01 00:00:00,1924-01-01,1924-01-01,16017.0,é-J. Drouhard,E. Drouhet,0.584
18410,Chaves,óM,NaN,NaN,NaN,NaN,NaN,NaN,óM Chaves,"Chaves,Ó.M., González,A.F.R. & Alfaro,E.",5,https://herbarium.bgbm.org/object/B101147146,2003-07-02 12:00:00,2003-07-02,2003-07-03,16018.0,óM Chaves,M. M. Chaves,0.918
18412,Černeva,Ž.,NaN,NaN,NaN,NaN,NaN,NaN,Ž. Černeva,"Markova,M. & Černeva,Ž.",1,https://herbarium.bgbm.org/object/B100310826,1975-04-19 00:00:00,1975-04-19,1975-04-19,16019.0,Ž. Černeva,O. Cherneva,0.523
18414,Fedorova,В.,NaN,NaN,NaN,NaN,NaN,NaN,В. Fedorova,"Tikhomirov,V. [Тихомиров,В.], Fedorova,T. & St...",1,https://herbarium.bgbm.org/object/B100630485,1991-06-04 00:00:00,1991-06-04,1991-06-04,16020.0,В. Fedorova,A. A. Fedorov,0.789


In [14]:
# join (only) full name matches with collector source data
collectors_matches_fullname = pd.merge(
    collectors_unique, matches_fullnames, 
    left_on='canonical_string_collector_parsed' , right_on='namematch_source_data'
    #, suffixes=(None, '_namematch') # append to left-data, right-data only when identical column names occur
)

collectors_matches_fullname # 429 rows × 15 columns

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
0,Hassan,Abdisalam Sheikh,NaN,NaN,NaN,NaN,NaN,NaN,Abdisalam Sheikh Hassan,"Friis,I., Vollesen,K. & Abdisalam Sheikh Hassan",7,https://herbarium.bgbm.org/object/B100003700,1987-05-30 14:24:00,1987-05-27,1987-06-14,0,Abdisalam Sheikh Hassan,Abdisalam S. Hassan,0.749
1,Hassan,Abdisalem Sheikh,NaN,NaN,NaN,NaN,NaN,NaN,Abdisalem Sheikh Hassan,"Friis,I, Vollesen,K., & Abdisalem Sheikh Hassan",1,https://herbarium.bgbm.org/object/B100003663,1987-06-07 00:00:00,1987-06-07,1987-06-07,1,Abdisalem Sheikh Hassan,Abdisalam S. Hassan,0.613
2,García-Mendoza,Abisaí J.,NaN,NaN,NaN,NaN,NaN,NaN,Abisaí J. García-Mendoza,"García-Mendoza,Abisaí J. & Ramos,C.",2,https://herbarium.bgbm.org/object/B100242996,1984-05-30 00:00:00,1981-08-08,1987-03-22,2,Abisaí J. García-Mendoza,Abisai Josue García-Mendoza,0.702
3,Hussain,Abrar,NaN,NaN,NaN,NaN,NaN,NaN,Abrar Hussain,"Abedin,S. & Abrar Hussain",1,https://herbarium.bgbm.org/object/B100240364,1972-05-26 00:00:00,1972-05-26,1972-05-26,3,Abrar Hussain,H. Hussain,0.673
4,Geheeb,Adalbert,NaN,NaN,NaN,NaN,NaN,NaN,Adalbert Geheeb,"Geheeb,Adalbert",1,https://dr.jacq.org/DR078137,1866-05-22 00:00:00,1866-05-22,1866-05-22,4,Adalbert Geheeb,Adalbert Geheeb,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,Chang,Zhao Y.,NaN,NaN,NaN,NaN,NaN,NaN,Zhao Y. Chang,"Chang, Zhao Y. (no. 2004-449)",21,http://id.snsb.info/snsb/collection/469321/575...,2004-05-24 14:31:34,2004-05-08,2004-06-01,338,Zhao Y. Chang,Zhao Yang Chang,0.659
339,Liu,Zheng-yu,NaN,NaN,NaN,NaN,NaN,NaN,Zheng-yu Liu,"Liu,Zheng-yu",1,https://herbarium.bgbm.org/object/B100517349,1996-03-05 00:00:00,1996-03-05,1996-03-05,339,Zheng-yu Liu,Zheng Yu Liu,1.000
340,Zhuo,Zhou,NaN,NaN,NaN,NaN,NaN,NaN,Zhou Zhuo,"Kilian,N., Raab-Straube,E. von, Zhang Jianwen,...",63,https://herbarium.bgbm.org/object/B100517202,2012-08-26 04:57:08,2012-08-17,2012-09-02,340,Zhou Zhuo,Shou Zhou Zhang,0.715
341,Heldreich,von,NaN,NaN,NaN,NaN,NaN,NaN,von Heldreich,"Heldreich,von",1,https://dr.jacq.org/DR058755,1873-01-01 00:00:00,1873-01-01,1873-01-01,341,von Heldreich,I. von Heldreich,0.886


In [15]:
# join all name matches together
collectors_all_matches=pd.concat([collectors_matches, collectors_matches_fullname])
collectors_all_matches.sort_values(by=['namematch_similarity', 'family'], ascending=[False, True], inplace=True)
collectors_all_matches.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
3136,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,A. Aaronsohn,"Aaronsohn,A.",1,https://herbarium.bgbm.org/object/B100379341,1908-06-20 00:00:00,1908-06-20,1908-06-20,2647.0,A. Aaronsohn,A. Aaronsohn,1.0
7256,Abbe,E.C.,NaN,NaN,NaN,NaN,NaN,NaN,E.C. Abbe,"Raup,H.M. & Abbe,E.C.",2,https://herbarium.bgbm.org/object/B100241637,1946-05-22 12:00:00,1932-07-26,1960-03-18,6300.0,E.C. Abbe,E. C. Abbe,1.0
11874,Abbott,J.R.,NaN,NaN,NaN,NaN,NaN,NaN,J.R. Abbott,"Bécquer,E. & Abbott,J.R.",80,https://herbarium.bgbm.org/object/B100181131,1999-11-05 15:33:58,1992-05-20,2007-07-17,10380.0,J.R. Abbott,J. R. Abbott,1.0
18150,Abbott,W.L.,NaN,NaN,NaN,NaN,NaN,NaN,W.L. Abbott,"Abbott, W.L. (no. 1248)",4,http://id.snsb.info/snsb/collection/504820/626...,1921-08-04 16:00:00,1921-04-05,1922-04-27,15816.0,W.L. Abbott,W. L. Abbott,1.0
16267,Abedin,S.,NaN,NaN,NaN,NaN,NaN,NaN,S. Abedin,"Abedin,S. & Hussein,A.",13,https://herbarium.bgbm.org/object/B100046632,1971-09-20 06:00:00,1969-05-29,1972-05-26,14189.0,S. Abedin,S. Abedin,1.0


In [16]:
# Save the plain name matching results only ...
## cell split - code
# if not os.path.exists('data'):
#     print("Make data directory for saving …")
#     os.makedirs('data')
# 
# # Set some global varialbes
# # this_timestamp_for_data=time.strftime('%Y%m%d') # 20230719
# this_timestamp_for_data=20231030
# 
# this_output_file='data/results_bgbm_collectors_vs_wikidata-botanists_cossim-similarity_plain-names_%s.csv' % (
#     this_timestamp_for_data
# )
# 
# collectors_all_matches.to_csv(this_output_file)
# 
# print("Wrote plain name matches of collector names into %s (%d kB)" % 
#     (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
# )
## cell split - code
# old code # Join Wikidata items
# df_avh_matches_wikidata = pd.merge(df_avh_matches, df_wikidata                , left_on='namematch_resource_data', right_on='canonical_string', how='left')
# df_avh_matches_wikidata = pd.merge(df_avh_matches_wikidata, df_wikidata_unique, left_on='namematch_resource_data', right_on='canonical_string', how='left')
# df_avh_matches_wikidata.rename(columns={df_avh_matches_wikidata.columns.tolist()[-1]: 'dup_count'}, inplace=True)


In [17]:
# merge now with WikiData: the matching data and the wiki data’s on the conaonical string name
collectors_matches_tmp_names_abbreviated = pd.merge(
    collectors_matches, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string'
)
collectors_matches_tmp_fullnames = pd.merge(
    collectors_matches_fullname, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string_fullname'
)
collectors_matches_g1_merged_wikidata = pd.concat(
    [collectors_matches_tmp_names_abbreviated, collectors_matches_tmp_fullnames]
    , ignore_index=True
)

In [18]:
print("Show example data of «Kotschy…» with Cosine Similiarity we had 0.6 … 1.0 (Nearest Neighbour distances were from 0.0 to almost 1.0)")
print("There was a match of Kotschyi, C.G.T. → Kotschy, T.   → 0.614 → http://www.wikidata.org/wiki/Q113299  with lower similarity and a correct match to Carl Georg Theodor … :-)")
print("There was a match of Kotschy, C.G.T.  → Kotschy, C.F. → 0.824 → http://www.wikidata.org/entity/Q86842 with higer similarity but it is probably a wrong match of Carl Georg Theodor → to Carl Friedrich … :-/")

criterion = collectors_matches_g1_merged_wikidata['canonical_string_collector_parsed'].str.contains('Kotschy')
collectors_matches_g1_merged_wikidata[criterion].get([
    # 'canonical_string_collector_parsed', # canonical_string_collector_parsed = namematch_source_data
    'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
    'namematch_source_data', 'namematch_resource_data', 'namematch_similarity', 
    # 'canonical_string_fullname', 
    'itemLabel', 'wikidata_link'
])

Show example data of «Kotschy…» with Cosine Similiarity we had 0.6 … 1.0 (Nearest Neighbour distances were from 0.0 to almost 1.0)
There was a match of Kotschyi, C.G.T. → Kotschy, T.   → 0.614 → http://www.wikidata.org/wiki/Q113299  with lower similarity and a correct match to Carl Georg Theodor … :-)
There was a match of Kotschy, C.G.T.  → Kotschy, C.F. → 0.824 → http://www.wikidata.org/entity/Q86842 with higer similarity but it is probably a wrong match of Carl Georg Theodor → to Carl Friedrich … :-/


,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,namematch_source_data,namematch_resource_data,namematch_similarity,itemLabel,wikidata_link
8480,1,http://id.snsb.info/snsb/collection/22980/3175...,C.G. Kotschy,T. Kotschy,0.811,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
8481,2215,http://id.snsb.info/snsb/collection/108230/167...,C.G.T. Kotschy,T. Kotschy,0.859,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
8482,1,https://herbarium.bgbm.org/object/B100160086,C.G.T. Kotschyi,T. Kotschy,0.859,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
8483,36,http://id.snsb.info/snsb/collection/16719/2549...,K.G.T. Kotschy,T. Kotschy,0.852,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
8484,294,http://id.snsb.info/snsb/collection/117808/176...,T. Kotschy,T. Kotschy,1.000,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
8486,3,https://herbarium.bgbm.org/object/B100526350,Th Kotschy,T. Kotschy,0.764,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
17381,1,https://herbarium.bgbm.org/object/B101086119,Carl Georg Theodor Kotschy,Theodor Kotschy,0.795,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299


In [19]:
pprint.pprint(collectors_matches_g1_merged_wikidata.columns)

Index(['family', 'given', 'suffix', 'particle', 'dropping_particle', 'nick',
       'appellation', 'title', 'canonical_string_collector_parsed',
       'source_data', 'occurrenceID_collectors_count',
       'occurrenceID_collectors_firstsample', 'collectors_eventDate_mean',
       'collectors_eventDate_min', 'collectors_eventDate_max', 'old_index',
       'namematch_source_data', 'namematch_resource_data',
       'namematch_similarity', 'item', 'itemLabel', 'surname', 'initials',
       'canonical_string', 'canonical_string_fullname', 'orcid', 'viaf',
       'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob', 'yod', 'wyb',
       'wye', 'wikidata_link', 'orcid_link', 'harv_link', 'ipni_link',
       'bionomia_link'],
      dtype='object')


In [20]:
collectors_matches_g1_merged_wikidata.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,...,bionomia_id,yob,yod,wyb,wye,wikidata_link,orcid_link,harv_link,ipni_link,bionomia_link
0,Aaiki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aaiki,Murashige & Aaiki,...,NaN,<NA>,<NA>,<NA>,<NA>,http://www.wikidata.org/wiki/Q33691780,NaN,NaN,https://www.ipni.org/a/20022012-1,NaN
1,Saiki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Saiki,"Degener,O., Murashige,T. & Saiki",...,NaN,<NA>,<NA>,<NA>,<NA>,http://www.wikidata.org/wiki/Q33691780,NaN,NaN,https://www.ipni.org/a/20022012-1,NaN
2,Abner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abner,Abner,...,NaN,1761,1826,<NA>,<NA>,http://www.wikidata.org/wiki/Q77459,NaN,NaN,NaN,NaN
3,Hübner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hübner,Hübner,...,NaN,1761,1826,<NA>,<NA>,http://www.wikidata.org/wiki/Q77459,NaN,NaN,NaN,NaN
4,Hübner,G.,NaN,NaN,NaN,NaN,NaN,NaN,G. Hübner,"Hübner,G.",...,NaN,1761,1826,<NA>,<NA>,http://www.wikidata.org/wiki/Q77459,NaN,NaN,NaN,NaN


In [23]:
this_timestamp_for_data=20231116
do_custom_data_aggregation=True
if do_custom_data_aggregation:
    # Select useful columns for data results
    collectors_wikidata_cossim = collectors_matches_g1_merged_wikidata[
        ['canonical_string_collector_parsed', 'family', 'given', 
         'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
         'source_data',
        'namematch_source_data', 'namematch_resource_data', 'namematch_similarity', 
        'item', 'canonical_string', 'itemLabel',
        'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob', 'yod', 'wyb']
    ]
    
    # Order by similarity (desc), number of Wikidata items (asc) and number of collections (desc)
    collectors_wikidata_cossim.sort_values(by=['namematch_similarity', 'family'], ascending=[False, True], inplace=True)
    
    display(collectors_wikidata_cossim)
    # cell split - code
    # Kotschy example again with all merged columns
    pd.set_option("display.max_columns", None) # default ?20
    
    criterion = collectors_wikidata_cossim['canonical_string_collector_parsed'].str.contains('Kotschy')
    print("Show example of «Kotschy…» with similarities of 0.5 … 1.0")
    print("There was a match of Kotschy, C.G.T. → Kotschy, C.F. → 0.824 → http://www.wikidata.org/entity/Q86842 and it is probably a wrong match of Carl Georg Theodor → to Carl Friedrich … :-/")
    display(collectors_wikidata_cossim[criterion])
    # cell split - code

    # TODO further evaluation or filtering, counting, clean up aso.
    if not os.path.exists('data'):
        os.makedirs('data')
    
    # bgbm_collectors_cosine-similarity_wikidata-botanists_%s.csv
    this_output_file='data/results_bgbm_collectors_vs_wikidata-botanists_cossim-similarity_merged-data_%s.csv' % (
        this_timestamp_for_data
    )
    
    collectors_wikidata_cossim.to_csv(this_output_file)
    
    print("Wrote matches of collector names into %s (%d kB)" % 
        (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
    )
    pd.set_option("display.max_columns", 20)

/tmp/ipykernel_42341/1766163911.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collectors_wikidata_cossim.sort_values(by=['namematch_similarity', 'family'], ascending=[False, True], inplace=True)


,canonical_string_collector_parsed,family,given,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,source_data,namematch_source_data,namematch_resource_data,namematch_similarity,item,...,orcid,viaf,isni,harv,ipni,abbr,bionomia_id,yob,yod,wyb
5693,A. Aaronsohn,Aaronsohn,A.,1,https://herbarium.bgbm.org/object/B100379341,"Aaronsohn,A.",A. Aaronsohn,A. Aaronsohn,1.0,http://www.wikidata.org/entity/Q2086130,...,NaN,2795076,0000 0001 0948 8581,30592,23-1,Aarons.,Q2086130,1876,1919,<NA>
9762,E.C. Abbe,Abbe,E.C.,2,https://herbarium.bgbm.org/object/B100241637,"Raup,H.M. & Abbe,E.C.",E.C. Abbe,E. C. Abbe,1.0,http://www.wikidata.org/entity/Q10274118,...,NaN,101473381,0000 0000 7237 8505,30066,26-1,Abbe,Q10274118,1905,2000,<NA>
13184,J.R. Abbott,Abbott,J.R.,80,https://herbarium.bgbm.org/object/B100181131,"Bécquer,E. & Abbott,J.R.",J.R. Abbott,J. R. Abbott,1.0,http://www.wikidata.org/entity/Q18982386,...,NaN,NaN,NaN,90638,20015671-1,J.R.Abbott,NaN,1968,<NA>,<NA>
17148,W.L. Abbott,Abbott,W.L.,4,http://id.snsb.info/snsb/collection/504820/626...,"Abbott, W.L. (no. 1248)",W.L. Abbott,W. L. Abbott,1.0,http://www.wikidata.org/entity/Q635604,...,NaN,1545420,0000 0000 3712 5377,27518,NaN,NaN,Q635604,1860,1936,<NA>
5030,S. Abedin,Abedin,S.,13,https://herbarium.bgbm.org/object/B100046632,"Abedin,S. & Hussein,A.",S. Abedin,S. Abedin,1.0,http://www.wikidata.org/entity/Q16142861,...,NaN,5859151837993620520007,NaN,69097,35239-1,Abedin,NaN,1952,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3436,Mortelmans,Mortelmans,NaN,1,https://herbarium.bgbm.org/object/B100160566,Mortelmans,Mortelmans,A. Morte,0.5,http://www.wikidata.org/entity/Q88808710,...,NaN,NaN,NaN,NaN,20034489-1,A.Morte,NaN,<NA>,<NA>,<NA>
10261,F. Rizo-Patrón,Rizo-Patrón,F.,2,https://herbarium.bgbm.org/object/B101144501,"Chavarría,U. & Rizo-Patrón.,F.",F. Rizo-Patrón,S. Rizo,0.5,http://www.wikidata.org/entity/Q27786423,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1760,1816,<NA>
17299,Z. Shuren,Shuren,Z.,1,https://herbarium.bgbm.org/object/B100275259,"Smirnov,S.V., German,D.A., Wenli,C., Shuren,Z....",Z. Shuren,L. Z. Shue,0.5,http://www.wikidata.org/entity/Q19002067,...,NaN,NaN,NaN,23938,15425-1,L.Z.Shue,NaN,<NA>,<NA>,<NA>
1029,I. Silvia,Silvia,I.,1,https://herbarium.bgbm.org/object/B100058014,"Greuter W., Rankin R., Ferro & Silvia,I.",I. Silvia,A. Silva,0.5,http://www.wikidata.org/entity/Q88809477,...,NaN,NaN,NaN,NaN,20034509-1,A.M.S.Silva,NaN,<NA>,<NA>,<NA>


Show example of «Kotschy…» with similarities of 0.5 … 1.0
There was a match of Kotschy, C.G.T. → Kotschy, C.F. → 0.824 → http://www.wikidata.org/entity/Q86842 and it is probably a wrong match of Carl Georg Theodor → to Carl Friedrich … :-/


,canonical_string_collector_parsed,family,given,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,source_data,namematch_source_data,namematch_resource_data,namematch_similarity,item,canonical_string,itemLabel,orcid,viaf,isni,harv,ipni,abbr,bionomia_id,yob,yod,wyb
8484,T. Kotschy,Kotschy,T.,294,http://id.snsb.info/snsb/collection/117808/176...,"Kotschy, T. (no. 119)",T. Kotschy,T. Kotschy,1.000,http://www.wikidata.org/entity/Q113299,T. Kotschy,Theodor Kotschy,NaN,5113711,0000 0000 8084 6890,23120,4989-1,Kotschy,Q113299,1813,1866,<NA>
8481,C.G.T. Kotschy,Kotschy,C.G.T.,2215,http://id.snsb.info/snsb/collection/108230/167...,"Kotschy, C.G.T. (no. 286)",C.G.T. Kotschy,T. Kotschy,0.859,http://www.wikidata.org/entity/Q113299,T. Kotschy,Theodor Kotschy,NaN,5113711,0000 0000 8084 6890,23120,4989-1,Kotschy,Q113299,1813,1866,<NA>
8482,C.G.T. Kotschyi,Kotschyi,C.G.T.,1,https://herbarium.bgbm.org/object/B100160086,"Kotschyi, C.G.T.",C.G.T. Kotschyi,T. Kotschy,0.859,http://www.wikidata.org/entity/Q113299,T. Kotschy,Theodor Kotschy,NaN,5113711,0000 0000 8084 6890,23120,4989-1,Kotschy,Q113299,1813,1866,<NA>
8483,K.G.T. Kotschy,Kotschy,K.G.T.,36,http://id.snsb.info/snsb/collection/16719/2549...,"Kotschy, K.G.T. (no. 97)",K.G.T. Kotschy,T. Kotschy,0.852,http://www.wikidata.org/entity/Q113299,T. Kotschy,Theodor Kotschy,NaN,5113711,0000 0000 8084 6890,23120,4989-1,Kotschy,Q113299,1813,1866,<NA>
8480,C.G. Kotschy,Kotschy,C.G.,1,http://id.snsb.info/snsb/collection/22980/3175...,"Kotschy, C.G. (no. 88)",C.G. Kotschy,T. Kotschy,0.811,http://www.wikidata.org/entity/Q113299,T. Kotschy,Theodor Kotschy,NaN,5113711,0000 0000 8084 6890,23120,4989-1,Kotschy,Q113299,1813,1866,<NA>
17381,Carl Georg Theodor Kotschy,Kotschy,Carl Georg Theodor,1,https://herbarium.bgbm.org/object/B101086119,"Kotschy,Carl (Karl) Georg Theodor",Carl Georg Theodor Kotschy,Theodor Kotschy,0.795,http://www.wikidata.org/entity/Q113299,T. Kotschy,Theodor Kotschy,NaN,5113711,0000 0000 8084 6890,23120,4989-1,Kotschy,Q113299,1813,1866,<NA>
8486,Th Kotschy,Kotschy,Th,3,https://herbarium.bgbm.org/object/B100526350,"Kotschy,Th.",Th Kotschy,T. Kotschy,0.764,http://www.wikidata.org/entity/Q113299,T. Kotschy,Theodor Kotschy,NaN,5113711,0000 0000 8084 6890,23120,4989-1,Kotschy,Q113299,1813,1866,<NA>


Wrote matches of collector names into data/results_bgbm_collectors_vs_wikidata-botanists_cossim-similarity_merged-data_20231116.csv (4185 kB)


## Output Mapping to DarwinCore Attribution Output

Here we map table data fields to fields of DarwinCore Attribution (<https://github.com/tdwg/attribution/>, <https://github.com/tdwg/attribution/blob/master/people/dwc/agent_actions_v2020-09-08.xml>) 

## Scoring

Individual scored properties should actually be balanced in such a way that one can simply add up these different property scores; in this case, assessment of the calculated values is still necessary. The problem here with calculation with a distance measure is that we have the opposite of similarity, whose distance can become greater than 1, which must somehow be mapped to a scope of 0 … 1 (or -1 … 0 … 1) (TODO review).

General thoughts: With a score of -1 to 1, it can be assumed that:
* -1 means full devaluation or no agreement
* 1 means full upvoting or agreement, and
* 0 can have several interpretations: it is in between, or no rating possible, or missing values.

### Task to Be Solved in Evaluating the Life Time ~ Rating/Scoring

We have grouped the collection date (evenDate) to the name in the source data, so it may be that for (abbreviated) names, e.g. “Bachmann, F.”, the collection date is valid for *several* personal names, not just one. This must be taken into account when considering and evaluating whether the life data match the collection date. The rating of the life data has the following idea:

| Score (life time) | Remarks | 
|--|--|
| 1.0  | complete match                     |
| 0.5  | somewhat correct, but has errors or mistakes, indicating multiple person names    |
| 0.0   | no evaluation (or not possible) |
| -0.5 | is rather to be rejected, indicating multiple person names and possibly overlapping time spans of the collection date of different person names, or mistakes in the original data |
| -1.0 | completely rejected                |

### Task to Be Solved With Several Names ~ Assessment/Score

Since we do not know if there are other possible names somewhere when there is only one name, we cannot assign a “1” (= full agreement) with certainty, so it was decided that if only 1 name was found, this would be evaluated as zero, in the sense of no evaluation. So when evaluating the multiple names, only the mismatches are evaluated, according to the idea:

| Score (multiple names) | Remarks | 
|--|--|
| 1.0  | this value (=full upvoting or agreement) would never be set in this regard, since we do not know all the full names of the cosmos ;-), and could state this score certainty of 1.0 |
| 0.0 | no evaluation, because only 1 name found | 
| less than 0 | multiple names found, i.e. deduction (perhaps just -0.5, as a decision needs to be made) | 

---

TODO review interpretation:

- the fields are defined in <https://github.com/tdwg/attribution/blob/master/people/dwc/agent_actions_v2020-09-08.xml> and regarding from this DwC-attribution concept: is it correct to map it like the following (`name` would represent the *interpreted* resource name (in long format), not the *source* collector `name` (in (theoretically) long format))?
    ```
    name          ← itemLabel (wikiData)
    alternateName ← canonical_string_collector_parsed (actual collector name)
    ```

In [24]:
pprint.pprint(collectors_matches_g1_merged_wikidata.columns)

Index(['family', 'given', 'suffix', 'particle', 'dropping_particle', 'nick',
       'appellation', 'title', 'canonical_string_collector_parsed',
       'source_data', 'occurrenceID_collectors_count',
       'occurrenceID_collectors_firstsample', 'collectors_eventDate_mean',
       'collectors_eventDate_min', 'collectors_eventDate_max', 'old_index',
       'namematch_source_data', 'namematch_resource_data',
       'namematch_similarity', 'item', 'itemLabel', 'surname', 'initials',
       'canonical_string', 'canonical_string_fullname', 'orcid', 'viaf',
       'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob', 'yod', 'wyb',
       'wye', 'wikidata_link', 'orcid_link', 'harv_link', 'ipni_link',
       'bionomia_link'],
      dtype='object')


In [25]:
# TODO review score (similarity + ((yob,yod) ~ (eventDate_min, eventDate_max))

collectors_wikidata_cossim = collectors_matches_g1_merged_wikidata[
    ['canonical_string_collector_parsed', 'family', 'given', 
     'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
     'source_data',
    'namematch_source_data', 'namematch_resource_data', 'namematch_similarity', 
    'item', 'canonical_string', 'itemLabel',
    'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 
    'collectors_eventDate_mean', 'collectors_eventDate_min', 'collectors_eventDate_max',
     'yob', 'yod', 'wyb'
    ]
]

# Order by similarity (desc), number of Wikidata items (asc) and number of collections (desc)
collectors_wikidata_cossim.sort_values(
    by=['namematch_similarity', 'family', 'given'], 
    ascending=[False, True, True], 
    inplace=True
)

dwcagent_attr_output=collectors_wikidata_cossim.get([
    "occurrenceID_collectors_firstsample", 
    "canonical_string_collector_parsed",
    'family', 'given',
    "namematch_similarity", 
    "source_data", 
    "itemLabel", 
    "item",
    "collectors_eventDate_min",
    "collectors_eventDate_max",
    'yob', 'yod'
]).copy()

dwcagent_attr_output['canonical_string_collector_parsed'].replace(to_replace=r'([^,]+),\s*(.+)', value='\\2 \\1', inplace=True, regex=True)
dwcagent_attr_output['namematch_similarity_annotation'] = dwcagent_attr_output['namematch_similarity'].astype(str).str.replace(r'(.+)', '\\1 (cos similarity)', regex=True)
# dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'namematch_similarity_annotation', '', allow_duplicates=True)

dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'life_time_periode', '', allow_duplicates=True)

combine_life_times = lambda this_df: ("%s-%s" % (this_df["yob"], this_df["yod"])).replace(r"<NA>", "?")
dwcagent_attr_output["life_time_periode"]=dwcagent_attr_output.apply(combine_life_times, axis="columns")

# dwcagent_attr_output["life_time_periode"]

years_from_birth_until_first_collection_activity = 10
dwcagent_attr_output["yob_is_lt_eventDate_min"] = dwcagent_attr_output["yob"] + years_from_birth_until_first_collection_activity < dwcagent_attr_output["collectors_eventDate_min"].dt.year
dwcagent_attr_output["yod_is_gt_eventDate_max"] = dwcagent_attr_output["yod"] > dwcagent_attr_output["collectors_eventDate_max"].dt.year
dwcagent_attr_output["custom_score_lifetime"] = 0
dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'custom_score_lifetime_annotation', '', allow_duplicates=True)

# df.loc[(df['column_of_interest'] … condition), 'fill_to_column'] = value 

dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"] & dwcagent_attr_output["yod_is_gt_eventDate_max"],
    "custom_score_lifetime"
] = 1
# True cases but <NA> missing values
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"] & dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull(),
    "custom_score_lifetime"
] = 1
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull() & dwcagent_attr_output["yod_is_gt_eventDate_max"],
    "custom_score_lifetime"
] = 1
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull() & dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull(),
    "custom_score_lifetime"
] = 0

# False cases
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == False),
    "custom_score_lifetime"
] = -1
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==True) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == False),
    "custom_score_lifetime"
] = 0.5
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == True),
    "custom_score_lifetime"
] = 0.5

# False cases but <NA> missing values
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull()),
    "custom_score_lifetime"
] = -0.5
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull()) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == False),
    "custom_score_lifetime"
] = -0.5

# annotations True cases
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"] & dwcagent_attr_output["yod_is_gt_eventDate_max"], 
    "custom_score_lifetime_annotation"
] = "full match"

# annotations True cases but <NA> missing values
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"] & dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull(), 
    "custom_score_lifetime_annotation"
] = "OK? year of death is missing"
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull() & dwcagent_attr_output["yod_is_gt_eventDate_max"], 
    "custom_score_lifetime_annotation"
] = "OK? year of birth is missing"
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull() & dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull(), 
    "custom_score_lifetime_annotation"
] = "unknown life time"

# annotations False cases
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == False), 
    "custom_score_lifetime_annotation"
] = "life time not matching any enventDate (yob + %s … yod)" % years_from_birth_until_first_collection_activity
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==True) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == False), 
    "custom_score_lifetime_annotation"
] = "OK yob + %s, but yod not matching, check name and liftime data" % years_from_birth_until_first_collection_activity
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == True), 
    "custom_score_lifetime_annotation"
] = "yob + %s not matching, OK yod, check name and liftime data" % years_from_birth_until_first_collection_activity
# annotations False cases but <NA> missing values
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull()), 
    "custom_score_lifetime_annotation"
] = "yob + %s not matching, yod unknown, check name and liftime data" % years_from_birth_until_first_collection_activity
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull()) & (dwcagent_attr_output["yod_is_gt_eventDate_max"]==False), 
    "custom_score_lifetime_annotation"
] = "yob unknown, yod not matching, check name and liftime data"

dwcagent_attr_output["custom_score_multiple_names"]=0 # 0 shall mean: we don’t know yet for real
dwcagent_attr_output.loc[
    (dwcagent_attr_output['canonical_string_collector_parsed'].duplicated(keep=False)),
    'custom_score_multiple_names'
] = -0.5 # one decision has to be made, so cut the range of -1 to 0 only into half (or include multiple count somehow?)

dwcagent_attr_output['custom_score_overall'] = (
    dwcagent_attr_output['namematch_similarity'] * \
    (
        ( dwcagent_attr_output["custom_score_lifetime"] + dwcagent_attr_output['custom_score_multiple_names']) / 2
    )
).round(3)

dwcagent_attr_output['attributionRemarks'] = dwcagent_attr_output.apply(
    lambda row: "{similarity_distance_note};"
                " {score_overall:.2f} (score overall);"
                " {lifetime_periode} (life time);"
                " {lifetime_score:.1f} (life time score);"
                " {lifetime_score_annote} (life time score note);"
                " {score_multinames:.2f} (score multiple names);"
        .format(
    similarity_distance_note=row['namematch_similarity_annotation'],
    lifetime_periode=row["life_time_periode"],
    lifetime_score=row["custom_score_lifetime"],
    lifetime_score_annote=row["custom_score_lifetime_annotation"],
    score_overall=row["custom_score_overall"],
    score_multinames=row["custom_score_multiple_names"]
    ), axis='columns'
)

# adjust dwcagent displayOrder also to olerall score
dwcagent_attr_output.sort_values(
    by=['namematch_similarity', 'family', 'given', 'custom_score_overall'], 
    ascending=[False, True, True, False], 
    inplace=True
)
# use ordered canonical_string_collector_parsed to generate displayOrder
temp_duplicated = dwcagent_attr_output['canonical_string_collector_parsed'].duplicated() 
    # duplicated() keeps the first value False and mark all other duplicats as True, i.e. we can cumulate the Trues, it gives the order index
temp_insert_value=temp_duplicated.groupby(dwcagent_attr_output['canonical_string_collector_parsed']).cumsum() + 1 # display order starts at 1, incrementing
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('canonical_string_collector_parsed') + 1, 'displayOrder', temp_insert_value, allow_duplicates=True)

# test an show example data
show_display_output=True
if show_display_output:
    display(dwcagent_attr_output.loc[dwcagent_attr_output['yob_is_lt_eventDate_min'] == True].get([
        # "occurrenceID_collectors_firstsample",
        "canonical_string_collector_parsed",
        'itemLabel',
        "custom_score_overall",
        "attributionRemarks",
        'custom_score_multiple_names',
        "namematch_similarity",
        # 'yob', 'yod',
        "life_time_periode", 
        'collectors_eventDate_min', 'collectors_eventDate_max', 
        "yob_is_lt_eventDate_min" ,'yod_is_gt_eventDate_max', 
        'custom_score_lifetime', 'custom_score_lifetime_annotation'
    ]).head(5))
    display(dwcagent_attr_output.loc[dwcagent_attr_output['yob_is_lt_eventDate_min'] == False].get([
        # "occurrenceID_collectors_firstsample",
        "canonical_string_collector_parsed",
        'itemLabel',
        "custom_score_overall",
        "attributionRemarks",
        'custom_score_multiple_names',
        "namematch_similarity",
        # 'yob', 'yod',
        "life_time_periode", 
        'collectors_eventDate_min', 'collectors_eventDate_max', 
        "yob_is_lt_eventDate_min" ,'yod_is_gt_eventDate_max', 
        'custom_score_lifetime', 'custom_score_lifetime_annotation'
    ]).head(5))

/tmp/ipykernel_42341/4017281334.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collectors_wikidata_cossim.sort_values(


,canonical_string_collector_parsed,itemLabel,custom_score_overall,attributionRemarks,custom_score_multiple_names,namematch_similarity,life_time_periode,collectors_eventDate_min,collectors_eventDate_max,yob_is_lt_eventDate_min,yod_is_gt_eventDate_max,custom_score_lifetime,custom_score_lifetime_annotation
5693,A. Aaronsohn,Aaron Aaronsohn,0.5,1.0 (cos similarity); 0.50 (score overall); 18...,0.0,1.0,1876-1919,1908-06-20,1908-06-20,True,True,1.0,full match
9762,E.C. Abbe,Ernst Cleveland Abbe,0.5,1.0 (cos similarity); 0.50 (score overall); 19...,0.0,1.0,1905-2000,1932-07-26,1960-03-18,True,True,1.0,full match
13184,J.R. Abbott,J. Richard Abbott,0.5,1.0 (cos similarity); 0.50 (score overall); 19...,0.0,1.0,1968-?,1992-05-20,2007-07-17,True,<NA>,1.0,OK? year of death is missing
17148,W.L. Abbott,William Louis Abbott,0.5,1.0 (cos similarity); 0.50 (score overall); 18...,0.0,1.0,1860-1936,1921-04-05,1922-04-27,True,True,1.0,full match
5030,S. Abedin,Sultanul Abedin,0.5,1.0 (cos similarity); 0.50 (score overall); 19...,0.0,1.0,1952-?,1969-05-29,1972-05-26,True,<NA>,1.0,OK? year of death is missing


,canonical_string_collector_parsed,itemLabel,custom_score_overall,attributionRemarks,custom_score_multiple_names,namematch_similarity,life_time_periode,collectors_eventDate_min,collectors_eventDate_max,yob_is_lt_eventDate_min,yod_is_gt_eventDate_max,custom_score_lifetime,custom_score_lifetime_annotation
14997,P. Aellen,Paul Aellen,0.25,1.0 (cos similarity); 0.25 (score overall); 18...,0.0,1.0,1896-1973,1848-04-28,1969-10-11,False,True,0.5,"yob + 10 not matching, OK yod, check name and ..."
15996,S. Ahmad,Shamsuddin Ahmad,-0.50,1.0 (cos similarity); -0.50 (score overall); 1...,-0.5,1.0,1942-?,1934-09-01,1957-01-15,False,<NA>,-0.5,"yob + 10 not matching, yod unknown, check name..."
14851,O. Anders,Olaf Anders,-0.25,1.0 (cos similarity); -0.25 (score overall); 1...,0.0,1.0,1931-?,1883-06-01,1973-08-30,False,<NA>,-0.5,"yob + 10 not matching, yod unknown, check name..."
116,Andrä,Édouard André,0.25,1.0 (cos similarity); 0.25 (score overall); 18...,0.0,1.0,1840-1911,1841-01-01,1895-10-01,False,True,0.5,"yob + 10 not matching, OK yod, check name and ..."
10429,O. Appel,Otto Appel,-0.75,1.0 (cos similarity); -0.75 (score overall); 1...,-0.5,1.0,1867-1952,1871-08-10,2017-08-10,False,False,-1.0,life time not matching any enventDate (yob + 1...


In [26]:
column_map_dwcagent_attr = {
    'occurrenceID_collectors_firstsample':'occurrenceID',
    'canonical_string_collector_parsed':  'alternateName',
    'source_data':                        'verbatimName',
    'itemLabel':                          'name',
    'item':                               'identifier',
    'collectors_eventDate_min':           'startedAtTime',
    'collectors_eventDate_max':           'endedAtTime',
    'namematch_similarity':               'custom_namematch_similarity'
}
dwcagent_attr_output.rename(
    mapper=column_map_dwcagent_attr,
    axis='columns',
    inplace=True)

dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('identifier')          + 1, 'agentIdentifierType', 'wikidata' , allow_duplicates=True)
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('agentIdentifierType') + 1, 'agentType'          , 'Person'   , allow_duplicates=True)
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('identifier')          + 1, 'action'             , 'collected', allow_duplicates=True)

show_display_output=False
if show_display_output:
    dwcagent_attr_output.head(20)

dwcagent_attr_output=dwcagent_attr_output.reindex(
    columns=[
        'occurrenceID', # no DwC agent standard (yet)?
        'verbatimName',
        'alternateName',
        'displayOrder', # shall start from 1, 2, 3 …
        'name',
        'attributionRemarks',
        'startedAtTime',
        'endedAtTime',
        'agentType',
        'action',
        'agentIdentifierType',
        'identifier',
        "custom_score_overall", # keep it for calculation convenience, no standard in DwC agent
        'custom_namematch_similarity',# keep it for calculation convenience, no standard in DwC agent
        'custom_score_multiple_names',# keep it for calculation convenience, no standard in DwC agent
        'custom_score_lifetime' # keep it for calculation convenience, no standard in DwC agent
    ]
)
# column deletion not neccessary after ….reindex(columns=[…])
# for this_column in ['yob', 'yod', 'life_time_periode', 'yob_is_lt_eventDate_min', 'yod_is_gt_eventDate_max', 'score_lifetime_annotation']:
#     del dwcagent_attr_output[this_column]


In [27]:
show_display_output=True
if show_display_output:
    # criterion = dwcagent_attr_output['alternateName'].map(lambda x: x.startswith('S. Ahmad'))
    criterion = dwcagent_attr_output['custom_score_multiple_names'].map(lambda this_score: this_score < 0 )  # show multiple name cases
    
    display(dwcagent_attr_output[criterion].head(20))

,occurrenceID,verbatimName,alternateName,displayOrder,name,attributionRemarks,startedAtTime,endedAtTime,agentType,action,agentIdentifierType,identifier,custom_score_overall,custom_namematch_similarity,custom_score_multiple_names,custom_score_lifetime
18,http://id.snsb.info/snsb/collection/444683/544...,"Ackermann, M. (no. 647)",M. Ackermann,1,Markus Ackermann,1.0 (cos similarity); 0.25 (score overall); 19...,2001-02-03,2022-10-06,Person,collected,wikidata,http://www.wikidata.org/entity/Q21504506,0.25,1.0,-0.5,1.0
19,http://id.snsb.info/snsb/collection/444683/544...,"Ackermann, M. (no. 647)",M. Ackermann,2,Marianne Ackermann,1.0 (cos similarity); -0.25 (score overall); ?...,2001-02-03,2022-10-06,Person,collected,wikidata,http://www.wikidata.org/entity/Q36619087,-0.25,1.0,-0.5,0.0
20,http://id.snsb.info/snsb/collection/444683/544...,"Ackermann, M. (no. 647)",M. Ackermann,3,Manfred Ackermann,1.0 (cos similarity); -0.25 (score overall); ?...,2001-02-03,2022-10-06,Person,collected,wikidata,http://www.wikidata.org/entity/Q47112660,-0.25,1.0,-0.5,0.0
16842,https://herbarium.bgbm.org/object/B101191388,"Adler,W.",W. Adler,1,Wolfgang Adler,1.0 (cos similarity); 0.25 (score overall); 19...,1992-06-12,2020-06-28,Person,collected,wikidata,http://www.wikidata.org/entity/Q119062956,0.25,1.0,-0.5,1.0
16841,https://herbarium.bgbm.org/object/B101191388,"Adler,W.",W. Adler,2,Wilhelm Adler,1.0 (cos similarity); -0.25 (score overall); ?...,1992-06-12,2020-06-28,Person,collected,wikidata,http://www.wikidata.org/entity/Q10393599,-0.25,1.0,-0.5,0.0
7767,https://herbarium.bgbm.org/object/B300230830,"Gradstein,S.R. & Aguirre,J.",J. Aguirre,1,Jaime Aguirre,1.0 (cos similarity); 0.25 (score overall); 19...,1980-08-06,1988-10-20,Person,collected,wikidata,http://www.wikidata.org/entity/Q21504658,0.25,1.0,-0.5,1.0
7768,https://herbarium.bgbm.org/object/B300230830,"Gradstein,S.R. & Aguirre,J.",J. Aguirre,2,Julio Aguirre,1.0 (cos similarity); -0.25 (score overall); ?...,1980-08-06,1988-10-20,Person,collected,wikidata,http://www.wikidata.org/entity/Q47112740,-0.25,1.0,-0.5,0.0
15997,https://herbarium.bgbm.org/object/B101097885,"Ahmad,S.",S. Ahmad,1,Sultan Ahmad,1.0 (cos similarity); 0.25 (score overall); 19...,1934-09-01,1957-01-15,Person,collected,wikidata,http://www.wikidata.org/entity/Q47112765,0.25,1.0,-0.5,1.0
15998,https://herbarium.bgbm.org/object/B101097885,"Ahmad,S.",S. Ahmad,2,Suhail Ahmad,1.0 (cos similarity); -0.25 (score overall); ?...,1934-09-01,1957-01-15,Person,collected,wikidata,http://www.wikidata.org/entity/Q88848655,-0.25,1.0,-0.5,0.0
15996,https://herbarium.bgbm.org/object/B101097885,"Ahmad,S.",S. Ahmad,3,Shamsuddin Ahmad,1.0 (cos similarity); -0.50 (score overall); 1...,1934-09-01,1957-01-15,Person,collected,wikidata,http://www.wikidata.org/entity/Q33661104,-0.50,1.0,-0.5,-0.5


In [28]:
# TODO further evaluation or filtering, counting, clean up aso.
if not os.path.exists('data'):
    os.makedirs('data')

# this_timestamp_for_data=time.strftime('%Y%m%d') # 20230719
this_timestamp_for_data=20231116
this_output_file='data/results_bgbm_collectors-eventDate_vs_wikidata-botanists_cossim-similarity_dwc-agent-output_%s.csv' % (
    this_timestamp_for_data
)

dwcagent_attr_output.to_csv(this_output_file, index=False)

print("Wrote matches of collector names as dwc-agent-output into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote matches of collector names as dwc-agent-output into data/results_bgbm_collectors-eventDate_vs_wikidata-botanists_cossim-similarity_dwc-agent-output_20231116.csv (6505 kB)


## Documentation

TODO document columns

Explanation of columns:

Column | Description
-|-
**Botanical collectors** |
family | parsed family name
given | parsed given name
suffix | suffix from name parsing
particle | particle from name parsing
dropping_particle | dropping_particle from name parsing
nick | nick name from name parsing
appellation | appellation from name parsing
title | title from name parsing
occurrenceID_collectors_count | count of all occurrenceID of one particular collector name
occurrenceID_collectors_firstsample | a data sample of an occurrenceID 
eventDate | date of the sampling event (required by GBIF, ☞ https://www.gbif.org/data-quality-requirements-sampling-events)
eventDate_min | calculated earliest date of all the sampling events within the data
eventDate_max | calculated latest date of all the sampling events within the data
eventDate_mean | calculated mean date of all the sampling events within the data
TODO activity_span | Number of years between first and last collection
**Name matching** |
namematch_source_data | matched name of the collector data set
namematch_resource_data | matched name of Wikidata the collector was tried to matched to
namematch_similarity | calculated cosine-similarity
**DarwinCore Agent Output** | (☞ [agent_actions_v2020-09-08.xml](https://github.com/tdwg/attribution/blob/master/people/dwc/agent_actions_v2020-09-08.xml))
occurrenceID | occurrence ID of the data item
name | the interpreted name match (https://github.com/tdwg/attribution/ The name of the item. In this case the *full name* as would be written on a legal document (without abbreviation), eg givenName familyName)
verbatimName | the source data name(s) (https://github.com/tdwg/attribution/ As written on occurrence, such as the collection or determination label.)
alternateName | the input name, collector source name (An alias for the item. Other full name agent may have been known under such as maiden name.)
displayOrder | I guess ordering the multiple name cases (https://github.com/tdwg/attribution/ The display order for the agent that executed the action when more than one agent was a participant.)
attributionRemarks | notes on the results (distance or similarity), including calculated value
agentType | The nature of the agent, e.g. "Person", "Organization", "SoftwareApplication"
action | The name of the single action written as a verb in past tense. Recommended best practice is to use a controlled vocabulary, examples "collected" or "identified"
agentIdentifierType | The type of identifier for the agent. (https://github.com/tdwg/attribution/ Recommended best practice is to use a controlled vocabulary, e.g. “ORCID”, “ISNI”, “Wikidata”, “VIAF”, “RoR”, “Ringgold”, “GRID”).
identifier | Wikidata ID (Recommended practice is to identify the resource by means of a string conforming to an identification system. Examples include International Standard Book Number (ISBN), Digital Object Identifier (DOI), and Uniform Resource Name (URN). Persistent identifiers should be provided as HTTP URIs.)
startedAtTime | (https://github.com/tdwg/attribution/ Start is when an action is deemed to have been started by an agent.) the first date of eventDate (supposedly the first sampling date), but grouped from collector name—in case of multiple name matches this first “sampling date” is less reliable and be reliable to relate to the source collector’s life time.
endedAtTime | (https://github.com/tdwg/attribution/ End is when an action is deemed to have been ended by an agent.) the last date of eventDate (supposedly the last sampling date), but grouped from collector name—in case of multiple name matches this first “sampling date” is less reliable and be reliable to relate to the source collector’s life time.
**Wikidata** |
item | Wikidata Item ID (URL)
itemLabel | Wikidata Item label (perhaps similar to the full name)
surname	| Surname; derived from item label
initials | Initials; derived from item label
canonical_string | Canonical name string; derived from item label, used for matching
orcid | ORCID ([P496](https://www.wikidata.org/wiki/Property:P496))
viaf | VIAF ID ([P214](https://www.wikidata.org/wiki/Property:P214))
isni | ISNI ID ([P213](https://www.wikidata.org/wiki/Property:P496))	
harv | Harvard Index of Botanists ID ([P6264](https://www.wikidata.org/wiki/Property:P6264))
ipni | IPNI author ID ([P586](https://www.wikidata.org/wiki/Property:P586))
abbr | botanist author abbreviation (standard form) ([P428](https://www.wikidata.org/wiki/Property:P428))
bionomia_id | identifier for a collector and/or determiner of natural history specimens, in the Bionomia database ([P6944](https://www.wikidata.org/wiki/Property:P6944))
yob	| Year of birth (derived from [P569](https://www.wikidata.org/wiki/Property:P569))
yod	| Year of death (derived from [P496](https://www.wikidata.org/wiki/Property:P570))
wyb	| Work year period begin ([P2031](https://www.wikidata.org/wiki/Property:P2031))
wye | Work year period end ([P2032](https://www.wikidata.org/wiki/Property:P2032))